# Witaj w generatorze checklist żeglarskich!

Naciśnij na górze przycisk "Run all". 

Następnie trzeba poczekać parę sekund, by zainstalowały się potrzebne pliki. 

Aplikacja jest gotowa, gdy poniżej pojawią się checkboxy do wyboru charakteru rejsu

In [ ]:
# @title
import os

if not os.path.exists('sailing-checklist'):
    !git clone https://github.com/SailNCode/sailing-checklist.git

In [ ]:
# @title
%pip install -q ipywidgets

Wybierz płeć:

In [ ]:
# @title
import ipywidgets as widgets
from IPython.display import display

gender_types = {
    "Kobieta": "f",
    "Mężczyzna": "m"
}
gender_choice = widgets.RadioButtons(
    options=list(gender_types.keys()),
    value=list(gender_types.keys())[0]
)

display(gender_choice)

Wybierz charakter rejsu (można kilka):

In [ ]:
# @title
types = {
    "always_get": "Podstawowy equipment",
    "mazury": "Mazury",
    "warm_sea": "Ciepłe morze",
    "foreign": "Zagraniczny",
    "individual": "Indywidualne rzeczy",
    "ice_boat": "Bojery - kompletna lista"
}
categories_choices = {}

for key, value in types.items():
  categories_choices[key] = widgets.Checkbox(description=value)
categories_choices["always_get"].value=True
display(*categories_choices.values())

Wygeneruj checklistę żeglarską poniżej:

In [ ]:
# @title
import json
from pathlib import Path

import ipywidgets as widgets
from IPython.display import Javascript, display

DIRECTORY = Path("sailing-checklist/categories")
SEPARATOR = "_"

def add_item(raw_item: str, category: str, target: dict, user_gender: str) -> None:
    parts = raw_item.split(SEPARATOR)

    if len(parts) == 1:
        target[category].append(raw_item)
        return

    item_name, item_gender = parts[0], parts[1]
    if user_gender == item_gender:
        target[category].append(item_name)


def load_category(category: str) -> dict | None:
    try:
        with open(DIRECTORY / f"{category}.json", "r") as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: There is no '{category}' list.")
        return None


def format_output(data: dict[str, list[str]]) -> str:
    lines = []
    for group, items in data.items():
        lines.append(group)
        for item in items:
            lines.append(f"\t{item}")
    return "\n".join(lines)

def on_generate_clicked(_):
    user_gender = gender_types[gender_choice.value]
    joined_list: dict[str, list[str]] = {}

    chosen_categories = [
        key for key, checkbox in categories_choices.items() if checkbox.value
    ]

    for category in chosen_categories:
        data = load_category(category)
        if not data:
            continue

        for name, item in data.items():
            joined_list.setdefault(name, [])

            if isinstance(item, str):
                add_item(item, name, joined_list, user_gender)

            elif isinstance(item, list):
                for list_item in item:
                    add_item(list_item, name, joined_list, user_gender)

    text_output.value = format_output(joined_list)


def on_copy_clicked(_):
    display(
        Javascript(
            f"navigator.clipboard.writeText(`{text_output.value}`);"
        )
    )

generate_button = widgets.Button(description="Wygeneruj")
generate_button.on_click(on_generate_clicked)

copy_button = widgets.Button(
    description="Kopiuj",
    button_style="success",
    icon="copy",
)
copy_button.on_click(on_copy_clicked)

text_output = widgets.Textarea(
    value="",
    placeholder="Tu pojawi się lista...",
    layout=widgets.Layout(width="100%", height="200px"),
)

display(widgets.VBox([text_output, widgets.HBox([generate_button, copy_button])]))
